In [1]:
import os
import json
import pandas as pd
from ultralytics import YOLO


In [2]:

# --- CONFIGURATION ---
# Chemin vers votre dossier principal
dataset_dir = "Data" 

# Chargement du modèle
# On utilise le modèle nano (léger) standard. 
# Il est entraîné sur COCO, donc on détectera la classe 0 ("person") qui correspond à la main/bras.
model = YOLO('yolov8n.pt') 

# Structures de données pour l'export
# Format JSON (Le plus important pour Edge Impulse CLI)
ei_labels = {
    "version": 1,
    "type": "bounding-box",
    "boundingBoxes": {}
}
# Format CSV (Pour votre demande spécifique)
csv_data = []

print(f"--- Analyse du dossier '{dataset_dir}' ---")

if not os.path.exists(dataset_dir):
    print(f"ERREUR: Le dossier '{dataset_dir}' est introuvable.")
    exit()

# Parcours des dossiers de classes (A.class, B.class, etc.)
for class_folder in sorted(os.listdir(dataset_dir)):
    folder_path = os.path.join(dataset_dir, class_folder)
    
    # Vérifie que c'est bien un dossier
    if not os.path.isdir(folder_path):
        continue
        
    # --- 1. NETTOYAGE DU LABEL ---
    # Transforme "A.class" en "A"
    if class_folder.endswith(".class"):
        label = class_folder.replace(".class", "")
    else:
        label = class_folder # Cas où le dossier serait déjà nommé "A"
    
    print(f"Traitement du dossier : {class_folder} -> Label : {label}")
    
    # Parcours des images (00000.jpg, 00001.jpg...)
    for image_file in os.listdir(folder_path):
        if not image_file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            continue
            
        full_path = os.path.join(folder_path, image_file)
        
        # --- IMPORTANT : GESTION DES DOUBLONS ---
        # Comme vos fichiers s'appellent tous 00000.jpg dans chaque dossier,
        # nous utilisons le chemin relatif (ex: "A.class/00000.jpg") comme clé unique.
        relative_path = os.path.join(class_folder, image_file).replace("\\", "/") # Force les slashs pour compatibilité
        
        # --- 2. INFERENCE YOLO ---
        # conf=0.4 : On garde les détections avec >40% de confiance
        # classes=[0] : On ne garde que la classe "personne" (votre main)
        results = model.predict(full_path, save=False, conf=0.4, classes=[0], verbose=False)
        
        for result in results:
            # On vérifie si une boîte a été trouvée
            if len(result.boxes) > 0:
                # On prend la boîte avec le meilleur score (la première)
                box = result.boxes[0]
                
                # Coordonnées YOLO (x_min, y_min, x_max, y_max)
                x_min, y_min, x_max, y_max = box.xyxy[0].tolist()
                
                # Conversion pour Edge Impulse (x, y, largeur, hauteur)
                x = int(x_min)
                y = int(y_min)
                w = int(x_max - x_min)
                h = int(y_max - y_min)
                
                # Ajout au JSON (Clé = chemin relatif)
                if relative_path not in ei_labels["boundingBoxes"]:
                    ei_labels["boundingBoxes"][relative_path] = []
                
                ei_labels["boundingBoxes"][relative_path].append({
                    "label": label,
                    "x": x,
                    "y": y,
                    "width": w,
                    "height": h
                })
                
                # Ajout au CSV
                csv_data.append([image_file, label, x, y, w, h, relative_path])

# --- 3. SAUVEGARDE DES FICHIERS ---

# Export JSON (Fichier clé pour l'import auto)
json_path = os.path.join(dataset_dir, "bounding_boxes.labels")
with open(json_path, 'w') as f:
    json.dump(ei_labels, f, indent=4)

# Export CSV
df = pd.DataFrame(csv_data, columns=['filename', 'label', 'x', 'y', 'width', 'height', 'path'])
csv_path = "bounding_boxes.csv"
df.to_csv(csv_path, index=False)

print("\n--- TERMINE ---")
print(f"1. Fichier JSON généré dans : {json_path}")
print(f"   (C'est ce fichier que Edge Impulse utilisera pour placer les boîtes automatiquement)")
print(f"2. Fichier CSV généré : {csv_path} (Pour votre vérification)")

--- Analyse du dossier 'Data' ---
Traitement du dossier : A.class -> Label : A
Traitement du dossier : B.class -> Label : B
Traitement du dossier : C.class -> Label : C
Traitement du dossier : D.class -> Label : D
Traitement du dossier : E.class -> Label : E
Traitement du dossier : F.class -> Label : F
Traitement du dossier : G.class -> Label : G
Traitement du dossier : H.class -> Label : H
Traitement du dossier : I.class -> Label : I
Traitement du dossier : K.class -> Label : K
Traitement du dossier : L.class -> Label : L
Traitement du dossier : M.class -> Label : M
Traitement du dossier : N.class -> Label : N
Traitement du dossier : O.class -> Label : O
Traitement du dossier : P.class -> Label : P
Traitement du dossier : Q.class -> Label : Q
Traitement du dossier : R.class -> Label : R
Traitement du dossier : S.class -> Label : S
Traitement du dossier : T.class -> Label : T
Traitement du dossier : U.class -> Label : U
Traitement du dossier : V.class -> Label : V
Traitement du dossier

In [1]:
import os
import json
import pandas as pd
from ultralytics import YOLO

# --- CONFIGURATION ---
dataset_dir = "Data"
model = YOLO("yolov8n.pt")

# === STRUCTURE CIBLE .labels (FORMAT files[]) ===
ei_labels = {
    "version": 1,
    "files": []
}

csv_data = []

print(f"--- Analyse du dossier '{dataset_dir}' ---")

if not os.path.exists(dataset_dir):
    raise FileNotFoundError(f"Dossier introuvable : {dataset_dir}")

# Parcours des dossiers de classes
for class_folder in sorted(os.listdir(dataset_dir)):
    folder_path = os.path.join(dataset_dir, class_folder)

    if not os.path.isdir(folder_path):
        continue

    # Nettoyage du label
    label = class_folder.replace(".class", "")

    print(f"📂 Dossier : {class_folder} → Label : {label}")

    for image_file in sorted(os.listdir(folder_path)):
        if not image_file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            continue

        full_path = os.path.join(folder_path, image_file)
        relative_path = f"{class_folder}/{image_file}".replace("\\", "/")

        # --- YOLO inference ---
        results = model.predict(
            full_path,
            conf=0.1,
            classes=[0],
            save=False,
            verbose=False
        )

        for result in results:
            if len(result.boxes) == 0:
                continue

            bounding_boxes = []

            for box in result.boxes:
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                x, y = int(x1), int(y1)
                w, h = int(x2 - x1), int(y2 - y1)

                bounding_boxes.append({
                    "label": label,
                    "x": x,
                    "y": y,
                    "width": w,
                    "height": h
                })

                csv_data.append([image_file, label, x, y, w, h, relative_path])

            # === AJOUT AU FORMAT files[] ===
            ei_labels["files"].append({
                "path": relative_path,
                "category": "training",
                "label": {
                    "type": "label",
                    "label": label
                },
                "metadata": {
                    "source": "yolov8-auto-label",
                    "model": "yolov8n"
                },
                "boundingBoxes": bounding_boxes
            })

# --- SAUVEGARDE ---

# Fichier .labels FINAL
labels_path = os.path.join(dataset_dir, "bounding_boxes.labels")
with open(labels_path, "w", encoding="utf-8") as f:
    json.dump(ei_labels, f, indent=2, ensure_ascii=False)

# CSV (optionnel)
df = pd.DataFrame(
    csv_data,
    columns=["filename", "label", "x", "y", "width", "height", "path"]
)
csv_path = os.path.join(dataset_dir, "bounding_boxes.csv")
df.to_csv(csv_path, index=False)

print("\n✅ TERMINÉ")
print(f"✔ Fichier .labels : {labels_path}")
print(f"✔ CSV : {csv_path}")


--- Analyse du dossier 'Data' ---
📂 Dossier : A.class → Label : A
📂 Dossier : B.class → Label : B
📂 Dossier : C.class → Label : C
📂 Dossier : D.class → Label : D
📂 Dossier : E.class → Label : E
📂 Dossier : F.class → Label : F
📂 Dossier : G.class → Label : G
📂 Dossier : H.class → Label : H
📂 Dossier : I.class → Label : I
📂 Dossier : K.class → Label : K
📂 Dossier : L.class → Label : L
📂 Dossier : M.class → Label : M
📂 Dossier : N.class → Label : N
📂 Dossier : O.class → Label : O
📂 Dossier : P.class → Label : P
📂 Dossier : Q.class → Label : Q
📂 Dossier : R.class → Label : R
📂 Dossier : S.class → Label : S
📂 Dossier : T.class → Label : T
📂 Dossier : U.class → Label : U
📂 Dossier : V.class → Label : V
📂 Dossier : W.class → Label : W
📂 Dossier : X.class → Label : X

✅ TERMINÉ
✔ Fichier .labels : Data\bounding_boxes.labels
✔ CSV : Data\bounding_boxes.csv


In [ ]:
import os
import json
import pandas as pd
from ultralytics import YOLO

# --- CONFIGURATION ---
dataset_dir = "Data"
model = YOLO("yolov8n.pt")

# === SÉLECTION INTERACTIVE DES LETTRES ===
print("📁 Dossiers trouvés dans Data/ :")
folders = []
for item in sorted(os.listdir(dataset_dir)):
    folder_path = os.path.join(dataset_dir, item)
    if os.path.isdir(folder_path):
        folders.append(item)
        print(f"  {len(folders)}. {item}")

print("\n" + "="*50)
print("CHOISISSEZ LES LETTRES À LABELLISER (ex: 1,3,5 ou 'all' pour toutes)")
print("Séparez par virgules : ", end="")

selection = input().strip().lower()

# Parsing de la sélection
if selection == "all":
    selected_folders = folders
else:
    try:
        indices = [int(x.strip()) - 1 for x in selection.split(",")]
        selected_folders = [folders[i] for i in indices if 0 <= i < len(folders)]
    except:
        print("❌ Sélection invalide. Utilisation de TOUTES les lettres.")
        selected_folders = folders

print(f"\n🎯 Lettres sélectionnées : {selected_folders}")

# === STRUCTURE CIBLE .labels (FORMAT files[]) ===
ei_labels = {
    "version": 1,
    "files": []
}

csv_data = []

print(f"\n--- Analyse des lettres sélectionnées ---")

# Parcours UNIQUEMENT des dossiers sélectionnés
for class_folder in selected_folders:
    folder_path = os.path.join(dataset_dir, class_folder)

    if not os.path.isdir(folder_path):
        print(f"⚠️  Dossier {class_folder} introuvable, ignoré.")
        continue

    # Nettoyage du label
    label = class_folder.replace(".class", "")

    print(f"📂 Traitement : {class_folder} → Label : {label}")

    img_count = 0
    box_count = 0

    for image_file in sorted(os.listdir(folder_path)):
        if not image_file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            continue

        img_count += 1
        full_path = os.path.join(folder_path, image_file)
        relative_path = f"{class_folder}/{image_file}".replace("\\", "/")

        # --- YOLO inference ---
        results = model.predict(
            full_path,
            conf=0.1,
            classes=[0],  # "person" = main
            save=False,
            verbose=False
        )

        for result in results:
            if len(result.boxes) == 0:
                continue

            bounding_boxes = []

            for box in result.boxes:
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                x, y = int(x1), int(y1)
                w, h = int(x2 - x1), int(y2 - y1)

                bounding_boxes.append({
                    "label": label,
                    "x": x,
                    "y": y,
                    "width": w,
                    "height": h
                })

                csv_data.append([image_file, label, x, y, w, h, relative_path])

            box_count += 1

            # === AJOUT AU FORMAT files[] ===
            ei_labels["files"].append({
                "path": relative_path,
                "category": "training",
                "label": {
                    "type": "label",
                    "label": label
                },
                "metadata": {
                    "source": "yolov8-auto-label",
                    "model": "yolov8n"
                },
                "boundingBoxes": bounding_boxes
            })

    print(f"   → {box_count}/{img_count} images labellisées")

# --- SAUVEGARDE ---

# Fichier .labels FINAL
labels_path = os.path.join(dataset_dir, "bounding_boxes.labels")
with open(labels_path, "w", encoding="utf-8") as f:
    json.dump(ei_labels, f, indent=2, ensure_ascii=False)

# CSV (optionnel)
if csv_data:
    df = pd.DataFrame(
        csv_data,
        columns=["filename", "label", "x", "y", "width", "height", "path"]
    )
    csv_path = os.path.join(dataset_dir, "bounding_boxes.csv")
    df.to_csv(csv_path, index=False)
    print(f"✔ CSV généré : {csv_path}")
else:
    print("⚠️  Aucun CSV généré (pas de détections YOLO)")

print(f"\n✅ TERMINÉ")
print(f"✔ Fichier .labels : {labels_path}")
print(f"✔ {len(ei_labels['files'])} images labellisées au total")


📁 Dossiers trouvés dans Data/ :
  1. A.class
  2. B.class
  3. C.class
  4. D.class
  5. E.class
  6. F.class
  7. G.class
  8. H.class
  9. I.class
  10. K.class
  11. L.class
  12. M.class
  13. N.class
  14. O.class
  15. P.class
  16. Q.class
  17. R.class
  18. S.class
  19. T.class
  20. U.class
  21. V.class
  22. W.class
  23. X.class

CHOISISSEZ LES LETTRES À LABELLISER (ex: 1,3,5 ou 'all' pour toutes)
Séparez par virgules : 

In [2]:
import os
import json
import cv2

# ========= CONFIG =========
DATASET_DIR = "Data"
LABELS_FILE = os.path.join(DATASET_DIR, "bounding_boxes.labels")

TARGET_LABEL = "A"   # 🔴 CHANGE ICI : "A", "B", "C", ...
WINDOW_NAME = "Edge Impulse Bounding Boxes"
BOX_COLOR = (0, 255, 0)   # Vert
TEXT_COLOR = (0, 255, 0)

# ==========================

if not os.path.exists(LABELS_FILE):
    raise FileNotFoundError(f"Fichier .labels introuvable : {LABELS_FILE}")

# --- Chargement du fichier .labels ---
with open(LABELS_FILE, "r", encoding="utf-8") as f:
    labels_data = json.load(f)

files = labels_data.get("files", [])

# --- Filtrage par lettre ---
filtered_files = [
    f for f in files
    if f.get("label", {}).get("label") == TARGET_LABEL
]

print(f"🔎 {len(filtered_files)} images trouvées pour la lettre '{TARGET_LABEL}'")

if not filtered_files:
    print("⚠️ Aucune image à afficher.")
    exit()

# --- Boucle d'affichage ---
for idx, item in enumerate(filtered_files, start=1):
    img_path = os.path.join(DATASET_DIR, item["path"])

    if not os.path.exists(img_path):
        print(f"❌ Image introuvable : {img_path}")
        continue

    img = cv2.imread(img_path)
    if img is None:
        print(f"❌ Impossible de lire : {img_path}")
        continue

    # Dessin des bounding boxes
    for box in item.get("boundingBoxes", []):
        x = int(box["x"])
        y = int(box["y"])
        w = int(box["width"])
        h = int(box["height"])
        lbl = box["label"]

        cv2.rectangle(
            img,
            (x, y),
            (x + w, y + h),
            BOX_COLOR,
            2
        )

        cv2.putText(
            img,
            lbl,
            (x, max(0, y - 8)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            TEXT_COLOR,
            2
        )

    # Infos affichées
    cv2.putText(
        img,
        f"{TARGET_LABEL}  ({idx}/{len(filtered_files)})",
        (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.9,
        (255, 0, 0),
        2
    )

    cv2.imshow(WINDOW_NAME, img)

    print(f"[{idx}/{len(filtered_files)}] {item['path']}")

    key = cv2.waitKey(0)

    # Quitter avec q ou ESC
    if key in [ord('q'), 27]:
        break

cv2.destroyAllWindows()


🔎 34 images trouvées pour la lettre 'A'
[1/34] A.class/00000.jpg
[2/34] A.class/00001.jpg
[3/34] A.class/00002.jpg
[4/34] A.class/00007.jpg
[5/34] A.class/00008.jpg
[6/34] A.class/00017.jpg
[7/34] A.class/00023.jpg
[8/34] A.class/00033.jpg
[9/34] A.class/00035.jpg
[10/34] A.class/00038.jpg
[11/34] A.class/00039.jpg
[12/34] A.class/00058.jpg
[13/34] A.class/00059.jpg
[14/34] A.class/00060.jpg
[15/34] A.class/00065.jpg
[16/34] A.class/00069.jpg
[17/34] A.class/00071.jpg
[18/34] A.class/00073.jpg
[19/34] A.class/00076.jpg
[20/34] A.class/00078.jpg
[21/34] A.class/00080.jpg
[22/34] A.class/00081.jpg
[23/34] A.class/00092.jpg
[24/34] A.class/00093.jpg
[25/34] A.class/00094.jpg
[26/34] A.class/00097.jpg
[27/34] A.class/00100.jpg
[28/34] A.class/00101.jpg
[29/34] A.class/00102.jpg
[30/34] A.class/00103.jpg
[31/34] A.class/00104.jpg
[32/34] A.class/00105.jpg
[33/34] A.class/00106.jpg
[34/34] A.class/00107.jpg


In [5]:
import csv
import os
from PIL import Image

CSV_FILE = "./Data/_annotations.csv"
IMAGE_ROOT = "Data"
OUTPUT_LABELS = "labels"

# mapping des classes
class_map = {
    "A": 0,
    "B": 1,
    "C": 2,
    "D": 3,
    "E": 4,
    "F": 5,
    "G": 6,
    "H": 7,
    "I": 8,
    "K": 9,
    "L": 10,
    "M": 11,
    "N": 12,
    "O": 13,
    "P": 14,
    "Q": 15,
    "R": 16,
    "S": 17,
    "T": 18,
    "U": 19,
    "V": 20,
    "W": 21,
    "X": 22,
    # ajouter d'autres classes ici
}

os.makedirs(OUTPUT_LABELS, exist_ok=True)

with open(CSV_FILE, newline='') as f:
    reader = csv.DictReader(f)
    for row in reader:
        image_path = os.path.join(IMAGE_ROOT, row["path"])
        img = Image.open(image_path)
        img_w, img_h = img.size

        # coordonnées depuis le coin haut-gauche
        x = float(row["x"])
        y = float(row["y"])
        w = float(row["width"])
        h = float(row["height"])

        # conversion vers centre YOLO
        x_center = (x + w / 2) / img_w
        y_center = (y + h / 2) / img_h
        w_norm = w / img_w
        h_norm = h / img_h

        class_id = class_map[row["label"]]

        label_file = os.path.join(
            OUTPUT_LABELS,
            row["filename"].replace(".jpg", ".txt")
        )

        with open(label_file, "a") as lf:
            lf.write(f"{class_id} {x_center} {y_center} {w_norm} {h_norm}\n")


In [3]:
print("Finis")

Finis


In [ ]:

"""
Auto-label ASL alphabet hands with YOLOv8 + MediaPipe Hands
Génère un fichier bounding_boxes.labels dans chaque sous-dossier (A, B)
compatible Edge Impulse (object detection).
"""
# cd /Users/mohammedaminebendaou/Downloads/
# source env_asl/bin/activate 
# python auto_label_yolo_cv.py 
import os
import json
import cv2
from ultralytics import YOLO
import mediapipe as mp


# ========= PARAMÈTRES GÉNÉRAUX =========
BASE_FOLDER = "./Data" #ASL_Alphabet_Dataset"
SUBFOLDERS = ["A","B","C","D","E","F","G","H","I","K"]#,"L","M","N","O","P","Q","R","S","T","U","V","W","X"] # classes / sous-dossiers
DEFAULT_CATEGORY = "split"       # non utilisé par Edge Impulse dans ce fichier

# Réglages YOLO (rappel élevé, petites mains)
YOLO_MODEL_PATH = "yolov8n.pt"   # ou un modèle plus gros si tu veux
CONF_PRIMARY   = 0.20            # seuil principal plus bas
CONF_FALLBACK  = 0.10            # second passage très permissif
IOU_NMS        = 0.40            # NMS moins agressif pour ne pas louper des mains
MAX_DET        = 2               # max 2 mains par image

# Réglages boîtes
MIN_REL_AREA   = 0.008           # surface min ~0.8% de l'image
MP_MARGIN      = 0.15            # marge autour de la main (15%)

# =======================================

# Charger YOLO
yolo = YOLO(YOLO_MODEL_PATH)

# MediaPipe Hands pour refinement / fallback
mp_hands = mp.solutions.hands
mp_detector = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.30
)


def clamp_box(x1, y1, x2, y2, W, H):
    """Force la boîte dans les bornes de l'image."""
    x1 = max(0.0, min(float(x1), float(W - 1)))
    y1 = max(0.0, min(float(y1), float(H - 1)))
    x2 = max(0.0, min(float(x2), float(W - 1)))
    y2 = max(0.0, min(float(y2), float(H - 1)))
    if x2 < x1:
        x1, x2 = x2, x1
    if y2 < y1:
        y1, y2 = y2, y1
    return x1, y1, x2, y2


def detect_with_yolo(img_path, W, H):
    """Renvoie une liste de boîtes (x1, y1, w, h) avec YOLO, ou [] si rien."""
    # Passage 1
    r = yolo(
        img_path,
        conf=CONF_PRIMARY,
        iou=IOU_NMS,
        max_det=MAX_DET,
        agnostic_nms=True
    )
    boxes = r[0].boxes

    # Passage 2 si vide
    if boxes is None or len(boxes) == 0:
        r = yolo(
            img_path,
            conf=CONF_FALLBACK,
            iou=IOU_NMS,
            max_det=MAX_DET,
            agnostic_nms=True
        )
        boxes = r[0].boxes

    dets = []
    if boxes is None:
        return dets

    for b in boxes:
        x1, y1, x2, y2 = b.xyxy[0].tolist()
        x1, y1, x2, y2 = clamp_box(x1, y1, x2, y2, W, H)
        w, h = x2 - x1, y2 - y1
        if w * h >= MIN_REL_AREA * W * H:
            dets.append((x1, y1, w, h))
    return dets


def refine_with_mediapipe(img_bgr, roi=None):
    """
    Utilise MediaPipe pour serrer la boîte autour de la main.
    - roi : (x1, y1, w, h) en pixels (optionnel). Si fourni, MediaPipe travaille
      dans cette région et renvoie des boîtes remappées dans le repère global.
    """
    H, W = img_bgr.shape[:2]

    if roi is not None:
        rx1, ry1, rw, rh = roi
        rx1, ry1, rx2, ry2 = clamp_box(rx1, ry1, rx1 + rw, ry1 + rh, W, H)
        crop = img_bgr[int(ry1):int(ry2), int(rx1):int(rx2)]
        if crop.size == 0:
            return []
        rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
        res = mp_detector.process(rgb)
        base_x, base_y = rx1, ry1
        crop_W, crop_H = crop.shape[1], crop.shape[0]
    else:
        rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        res = mp_detector.process(rgb)
        base_x, base_y = 0.0, 0.0
        crop_W, crop_H = W, H

    dets = []
    if res and res.multi_hand_landmarks:
        for lm in res.multi_hand_landmarks:
            xs = [p.x * crop_W for p in lm.landmark]
            ys = [p.y * crop_H for p in lm.landmark]
            x1, y1 = min(xs), min(ys)
            x2, y2 = max(xs), max(ys)

            # Ajoute une petite marge pour ne pas couper les doigts
            mx = MP_MARGIN * (x2 - x1)
            my = MP_MARGIN * (y2 - y1)
            gx1, gy1, gx2, gy2 = clamp_box(
                base_x + x1 - mx,
                base_y + y1 - my,
                base_x + x2 + mx,
                base_y + y2 + my,
                W, H
            )
            w, h = gx2 - gx1, gy2 - gy1
            if w * h >= MIN_REL_AREA * W * H:
                dets.append((gx1, gy1, w, h))

    return dets


for sub in SUBFOLDERS:
    folder = os.path.join(BASE_FOLDER, sub)
    if not os.path.isdir(folder):
        print(f"⚠️ Dossier introuvable: {folder}")
        continue

    print(f"📂 Traitement du dossier: {folder}")

    bb_map = {}
    total, with_box = 0, 0

    for img_name in os.listdir(folder):
        if not img_name.lower().endswith((".jpg", ".jpeg", ".png")):
            continue
        total += 1

        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue
        H, W = img.shape[:2]

        # 1) YOLO pour proposer 0–MAX_DET ROI
        yolo_dets = detect_with_yolo(img_path, W, H)

        # 2) MediaPipe sur chaque ROI pour serrer la boîte
        final_dets = []
        if yolo_dets:
            for roi in yolo_dets:
                refined = refine_with_mediapipe(img, roi=roi)
                if refined:
                    final_dets.extend(refined)
                else:
                    # fallback: garder la boîte YOLO si MP ne voit rien
                    final_dets.append(roi)

        # 3) Fallback global MediaPipe si encore aucune boîte
        if not final_dets:
            mp_dets = refine_with_mediapipe(img, roi=None)
            final_dets.extend(mp_dets)

        # 4) Option: limiter le nombre final de boîtes par image (ex. 2)
        if final_dets:
            final_dets = final_dets[:MAX_DET]
            with_box += 1
            bb_map[img_name] = [
                {
                    "label": sub,  # classe OD = nom du sous-dossier (A ou B)
                    "x": float(x),
                    "y": float(y),
                    "width": float(w),
                    "height": float(h)
                }
                for (x, y, w, h) in final_dets
            ]

    # Écriture du bounding_boxes.labels pour ce dossier
    out_path = os.path.join(folder, "bounding_boxes.labels")
    with open(out_path, "w") as f:
        json.dump(
            {
                "version": 1,
                "type": "bounding-box-labels",
                "boundingBoxes": bb_map
            },
            f,
            indent=2,
            ensure_ascii=False
        )

    print(f"✅ {sub}: {with_box}/{total} images avec boîtes -> {out_path}")


📂 Traitement du dossier: ./Data\A.class

image 1/1 c:\Users\bilaaaaaaal\Documents\ENSEEIHT\S9\NN_embedded_systems\Data\A.class\00000.jpg: 480x640 1 person, 96.0ms
Speed: 5.0ms preprocess, 96.0ms inference, 13.8ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 c:\Users\bilaaaaaaal\Documents\ENSEEIHT\S9\NN_embedded_systems\Data\A.class\00001.jpg: 480x640 1 person, 10.6ms
Speed: 1.8ms preprocess, 10.6ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 c:\Users\bilaaaaaaal\Documents\ENSEEIHT\S9\NN_embedded_systems\Data\A.class\00002.jpg: 480x640 1 person, 10.6ms
Speed: 1.8ms preprocess, 10.6ms inference, 4.2ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 c:\Users\bilaaaaaaal\Documents\ENSEEIHT\S9\NN_embedded_systems\Data\A.class\00003.jpg: 480x640 1 hot dog, 13.7ms
Speed: 1.4ms preprocess, 13.7ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 c:\Users\bilaaaaaaal\Documents\ENSEEIHT\S9\NN_embedded_systems\Data

c:\Users\bilaaaaaaal\anaconda3\envs\conda_3_11\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


image 1/1 c:\Users\bilaaaaaaal\Documents\ENSEEIHT\S9\NN_embedded_systems\Data\A.class\00068.jpg: 480x640 1 person, 10.4ms
Speed: 2.5ms preprocess, 10.4ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 c:\Users\bilaaaaaaal\Documents\ENSEEIHT\S9\NN_embedded_systems\Data\A.class\00069.jpg: 480x640 1 person, 7.6ms
Speed: 1.5ms preprocess, 7.6ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 c:\Users\bilaaaaaaal\Documents\ENSEEIHT\S9\NN_embedded_systems\Data\A.class\00070.jpg: 480x640 (no detections), 8.9ms
Speed: 1.7ms preprocess, 8.9ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 c:\Users\bilaaaaaaal\Documents\ENSEEIHT\S9\NN_embedded_systems\Data\A.class\00070.jpg: 480x640 1 person, 1 banana, 12.5ms
Speed: 1.7ms preprocess, 12.5ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 c:\Users\bilaaaaaaal\Documents\ENSEEIHT\S9\NN_embedded_systems\Data\A.class\00071.jpg: 480x640 1 